# Quantum Neural Network Classifiers: A Tutorial

## 项目介绍

该论文主要讨论了在量子神经网络的不同编码结构和编码策略在解决监督学习任务时的区别。文章对幅度编码和块编码在量子神经网络处理二分类问题时的表现进行了对比。作为测试基准，Fashion MNIST、MNIST、对称性保护拓扑态等数据集用来作为识别训练任务。本次论文复现要求：研究并理解文章内容,利用MindQuantum实现对Table 2中 对Fashion Mnist及MNIST的分类任务的结果。原文链接：https://scipost.org/SciPostPhysLectNotes.61

## 复现过程

### 数据预处理
原文章主要讨论的是利用量子神经网络在二分类问题中的应用，具体任务设置为对Fashion MNIST中对靴子和T恤图片进行分类，对MNIST中的手写体“0”和“1”进行分类。初始数据为.mat格式，想要用于训练我们需要对其进行预处理

In [2]:
import numpy as np                                          
from mindquantum.core.circuit import Circuit                
from mindquantum.core.gates import H, RX, RY, RZ,X    
from mindquantum.core.parameterresolver import PRGenerator  
from mindquantum.simulator import Simulator
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split   
from mindquantum.algorithm.library import amplitude_encoder
from mindquantum.algorithm.nisq import HardwareEfficientAnsatz     
from mindquantum.core.operators import QubitOperator           # 导入QubitOperator模块，用于构造泡利算符
from mindquantum.core.operators import Hamiltonian             # 导入Hamiltonian模块，用于构建哈密顿量
import mindspore as ms                                                                         # 导入mindspore库并简写为ms
from mindquantum.framework import MQLayer,MQN2Layer                                              # 导入MQLayer
# 导入HardwareEfficientAnsatz
from mindquantum.core.gates import RY           
import torch
from torchvision import datasets, transforms# 导入量子门RY
from scipy.ndimage import zoom
import random

/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/simulator/__init__.py:17: UserWarning: Unable import mqvector gpu backend due to: cannot import name '_mq_vector_gpu' from partially initialized module 'mindquantum' (most likely due to a circular import) (/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/__init__.py)
  from .available_simulator import SUPPORTED_SIMULATOR


In [9]:
# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor()
])
np.random.seed(10)
def filter_3_and_6(data):
    images, labels = data
    mask = (labels == 3) | (labels == 6)
    return images[mask], labels[mask]

# 下载和加载 MNIST 数据集
mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
filtered_data = filter_3_and_6((mnist_dataset.data, mnist_dataset.targets))
X_data, y = filtered_data  # X 图像数据 y 标签

# 分别对标签为 3 和 6 的数据进行随机抽样
def sample_data(X, y, label, sample_ratio=0.2):
    label_mask = (y == label)
    X_label = X[label_mask]
    y_label = y[label_mask]
    
    sample_size = int(len(y_label) * sample_ratio)
    sample_indices = np.random.choice(len(y_label), sample_size, replace=False)
    
    return X_label[sample_indices], y_label[sample_indices]

X_data_3, y_data_3 = sample_data(X_data, y, label=3, sample_ratio=0.1)
X_data_6, y_data_6 = sample_data(X_data, y, label=6, sample_ratio=0.1)

# 合并抽样后的数据
X_sampled = torch.cat((X_data_3, X_data_6), dim=0)
y_sampled = torch.cat((y_data_3, y_data_6), dim=0)

Compressed_X = np.array([zoom(img,0.4) for img in X_sampled])
Compressed_X = Compressed_X/255

print(Compressed_X.shape, y_sampled.shape)

(1204, 11, 11) torch.Size([1204])


In [10]:
X_train, X_test, y_train, y_test = train_test_split(Compressed_X, y_sampled, test_size=0.2, random_state=0, shuffle=True) # 将数据集划分为训练集和测试集
y_train[y_train==3]=1
y_train[y_train==6]=0
y_test[y_test==3]=1
y_test[y_test==6]=0
print(X_train.shape)                                                                                   # 打印训练集中样本的数据类型
print(X_test.shape)                                                                                    # 打印测试集中样本的数据类型

(963, 11, 11)
(241, 11, 11)


train_pixels和test_pixels分别为训练和测试用的像素数据，train_index和test_index为训练和测试标签。训练样本和测试样本大小分别为1000和200.

可以通过matplotlib.pyplot对样本进行可视化

目前训练数据为像素数据，不能作为输入态直接用于量子线路，需要用幅度编码将像素数据转换为线路参量数据.

mindquantum提供的mindquantum.algorithm.library.amplitude_encoder编码器可以方便地实现这一过程。

In [12]:
from mindquantum.algorithm.library import amplitude_encoder
def amplitude_param(pixels):
    param_rd = []
    _, parameterResolver = amplitude_encoder(pixels, 7)   
    for _, param in parameterResolver.items():
        param_rd.append(param)
    param_rd = np.array(param_rd)
    return param_rd

# 将幅度转为编码线路参数，幅度shape(256,)，参数shape(255,)
train_param = np.array([amplitude_param(i.flatten()) for i in X_train])
test_param = np.array([amplitude_param(i.flatten()) for i in X_test])

将参数数据和标签封装ms.dataset.NumpySlicesDataset用于训练，数据预处理环节完成。

该过程在main.py中Main()类的data_preporcess方法实现。

In [17]:
import mindspore as ms
BATCH_SIZE = 100
train_loader = ms.dataset.NumpySlicesDataset(
    {'features': train_param, 'labels': y_train}).batch(BATCH_SIZE) 
test_loader = ms.dataset.NumpySlicesDataset(
    {'features': test_param, 'labels': y_test}).batch(BATCH_SIZE)  

In [15]:
train_loader

TypeError: 'BatchDataset' object is not subscriptable

### 量子线路搭建

要实现数据分类需要定义含参ansatz线路。本次复现论文中提供三种ansatz，每个ansatz中包括参数层和纠缠层。

一层参数层和一层纠缠层构成一个复合层(block)，复合层数越多说明ansatz线路越深。

纠缠层有三种Ent1、Ent2、Ent3，所以三种ansatz对应的是三种纠缠层与参数层的组合。

In [8]:
from mindquantum.core.circuit import Circuit
import mindquantum.core.gates as Gate

def Classifying_circuit(qubit_num, block_num, Ent_circ):
    num = qubit_num
    depth = block_num
    circ = Circuit()
    for i in range(depth):
        circ = Para_circuit(circ, num)
        if Ent_circ == 'Ent1':
            circ = Ent1_circuit(circ, num)
        elif Ent_circ == 'Ent2':
            circ = Ent2_circuit(circ, num)
        elif Ent_circ == 'Ent3':
            circ = Ent3_circuit(circ, num)
    return circ

def Ent1_circuit(circuit,qubit_num):
    for i in range(0,qubit_num-1,2):
        circuit += Gate.Z.on(i+1,i)
    for i in range(1,qubit_num-2,2):
        circuit += Gate.Z.on(i+1,i)
    return circuit

def Ent2_circuit(circuit,qubit_num):
    for i in range(0,qubit_num-1,2):
        circuit += Gate.X.on(i+1,i)
    for i in range(1,qubit_num-2,2):
        circuit += Gate.X.on(i+1,i)
    return circuit

def Ent3_circuit(circuit,qubit_num):
    circuit = Ent2_circuit(circuit,qubit_num)
    circuit = Ent2_circuit(circuit,qubit_num)
    return circuit

def Para_circuit(circuit,qubit_num):
    for i in range(qubit_num):
        circuit += Gate.RX(f'Xtheta{i}').on(i)
        circuit += Gate.RZ(f'Ztheta{i}').on(i)
        circuit += Gate.RX(f'Xtheta2{i}').on(i)
    return circuit

Classifying_circuit(qubit_num, block_num, Ent_circ)可以通过传入比特数、复合层层数、纠缠层名称来定义ansatz

In [20]:
QUBIT_NUM = 10
BLOCK_NUM = 2
Ent = 'Ent3'
ansatz = Classifying_circuit(QUBIT_NUM,BLOCK_NUM,Ent).as_ansatz()

传入初态和比特数定义编码层

In [21]:
encoder = amplitude_encoder([0], QUBIT_NUM)[0].as_encoder()

In [ ]:
encoder.parameterized

随后通过Pauli-Z测量最后两个比特用于二分类，设置运行环境和全局种子后线路搭建完成。

In [22]:

from mindquantum.framework import MQLayer
from mindquantum.core.operators import Hamiltonian, QubitOperator
from mindquantum.simulator import Simulator
import mindspore as ms
WORKER = 4
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(1)
circ = encoder.as_encoder() + ansatz
meas = [Hamiltonian(QubitOperator(f'Z{i}')) for i in [QUBIT_NUM-2, QUBIT_NUM-1]]
sim = Simulator('mqvector', circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(meas, circ, parallel_worker=WORKER)
Qnet = MQLayer(grad_ops)

### 模型训练

In [23]:
class ForwardAndLoss(ms.nn.Cell):
    def __init__(self, backbone, loss_fn):
        super(ForwardAndLoss, self).__init__(auto_prefix=False)
        self.backbone = backbone
        self.loss_fn = loss_fn

    def construct(self, data, label):
        output = self.backbone(data)
        return self.loss_fn(output, label)

    def backbone_network(self):
        return self.backbone


class TrainOneStep(ms.nn.TrainOneStepCell):

    def __init__(self, network, optimizer):
        super(TrainOneStep, self).__init__(network, optimizer)
        self.grad = ms.ops.GradOperation(get_by_list=True)

    def construct(self, data, label):
        weights = self.weights
        loss = self.network(data, label)
        grads = self.grad(self.network, weights)(data, label)
        return loss, self.optimizer(grads)

LR = 0.05
loss = ms.nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean') 
opt = ms.nn.Adam(Qnet.trainable_params(), learning_rate=LR) 
net_with_loss = ForwardAndLoss(Qnet, loss)
train_one_step = TrainOneStep(net_with_loss, opt)

定义评价标准

In [24]:
class EpochLoss(ms.nn.Metric):
    def __init__(self):
        super(EpochLoss, self).__init__()
        self.clear()

    def clear(self):
        self.loss = 0
        self.counter = 0

    def update(self, *loss):
        loss = loss[0].asnumpy()
        self.loss += loss 
        self.counter += 1

    def eval(self):
        return self.loss / self.counter

class EpochAcc(ms.nn.Metric):
    def __init__(self):
        super(EpochAcc, self).__init__()
        self.clear()

    def clear(self):
        self.correct_num = 0
        self.total_num = 0

    def update(self, *inputs):
        y_output = inputs[0].asnumpy()
        y = inputs[1].asnumpy()
        y_pred = np.zeros_like(y)
        for i in range(y_pred.shape[0]):
            yi = y_output[i]
            if yi[0] >= yi[1]:
                y_pred[i] = 0
            else:
                y_pred[i] = 1       
        self.correct_num += np.sum(y == y_pred)
        self.total_num += y.shape[0] 

    def eval(self):
        return self.correct_num / self.total_num
        
acc_epoch = EpochAcc() 
loss_epoch = EpochLoss() 

开始训练

In [ ]:
train_loss_epoch = []
train_acc_epoch = []
test_loss_epoch = []
test_acc_epoch = []

STEP_NUM = 30

for epoch in range(STEP_NUM):
    loss_epoch.clear() 
    acc_epoch.clear()
    loss_epoch.clear()
    acc_epoch.clear()

    for data in train_loader: 
        train_one_step(data[0], data[1])   # 执行训练，并更新权重, data[0]参数，data[1]为标签
        loss = net_with_loss(data[0], data[1])  
        loss_epoch.update(loss) 
    train_loss = loss_epoch.eval() 
    train_loss_epoch.append(train_loss)

    # training accuracy
    for data in train_loader:
        logits = Qnet(data[0]) # 向前传播得到预测值
        acc_epoch.update(logits, data[1]) # 计算预测准确率
    train_acc = acc_epoch.eval()
    train_acc_epoch.append(train_acc)

    # testing loss
    for data in test_loader:
        loss = net_with_loss(data[0], data[1])  # 计算损失值
        loss_epoch.update(loss)
    test_loss = loss_epoch.eval()
    test_loss_epoch.append(test_loss)

    # testing accuracy
    for data in test_loader:
        logits = Qnet(data[0])
        acc_epoch.update(logits, data[1])
    test_acc = acc_epoch.eval()
    test_acc_epoch.append(test_acc)

    print(f"epoch: {epoch+1}, training loss: {train_loss}, training acc: {train_acc}, testing loss: {test_loss}, testing acc: {test_acc}")

In [ ]:
y

## 项目总结

原文中所给出的ansatz线路对FashionMNIST数据集中的二分类问题解析能力明显优于MNIST数据集。

在FashionMNIST分类中，基于控制Z门的纠缠层Ent1表现优于基于控制X门的Ent2和Ent3；而在MNIST分类中，Ent2和Ent3表现优于Ent1，此结论与原文规律吻合。